In [1]:
import os
import numpy as np
import pandas as pd
from tqdm import tqdm
from gmm import GMM_vl

%load_ext autoreload
%autoreload 2

In [2]:
df = pd.read_csv("../datasets/2B/consolidated_train.csv")
X = df.drop(["class","image", "index"], axis=1).to_numpy()
print(X.shape)
df.head()

(44280, 23)


,index,0,1,2,3,4,5,6,7,8,...,15,16,17,18,19,20,21,22,class,image
0,0,0.98080,0.98211,0.41813,1.014800e-11,2.590300e-10,3.115900e-07,-0.29073,-0.398550,-1.38820,...,0.003981,0.018343,1.6161,1.5720,1.8232,3.2124,2.2108,2.6424,0,0
1,1,0.98525,0.98722,0.55761,6.553400e-12,2.553300e-10,2.662800e-07,0.24050,0.047054,0.39490,...,0.005317,0.018217,1.6269,1.5189,1.8023,3.1327,3.0535,3.3129,0,0
2,2,0.98259,0.98731,0.66889,3.203300e-11,4.204600e-08,3.426400e-08,0.65259,-0.453870,0.22037,...,0.004586,0.018696,1.6867,1.7450,1.9101,2.4576,2.2328,2.5617,0,0
3,3,0.75408,0.91789,0.75235,8.701900e-03,2.167600e-06,4.994600e-08,-1.29070,0.170030,0.26758,...,0.002772,0.021064,2.7817,2.4161,1.5875,3.5543,3.2414,2.6642,0,0
4,4,0.28608,0.85314,0.80646,1.554800e-03,6.808300e-07,3.509400e-08,-0.58280,-1.183300,1.12540,...,0.006324,0.008592,2.5858,1.5805,2.4957,2.9218,2.8988,3.1211,0,0


In [5]:
gmm = GMM_vl(q=14)
df_selected = df[df["class"]==0]

X_selected = df_selected.drop(["class", "image", "index"], axis=1).to_numpy()
gmm.fit(X_selected, epochs=2)

 50%|█████     | 1/2 [04:36<04:36, 276.99s/it]

Difference is less than 0


In [4]:
gmm.probab(df_selected)

LinAlgError: singular matrix

In [7]:
gmm.gamma

array([[8.38623026e-019, 4.38163896e-187, 1.77915741e-005, ...,
        9.99973152e-001, 1.87610563e-093, 5.42385953e-011],
       [7.03975433e-017, 9.35616739e-193, 8.46677694e-005, ...,
        9.98620695e-001, 4.21012433e-163, 2.29703504e-015],
       [3.63558071e-017, 7.29250686e-185, 1.98693589e-011, ...,
        9.99411556e-001, 6.57943793e-145, 9.25386909e-015],
       ...,
       [1.54744424e-015, 8.52911841e-063, 6.45816649e-022, ...,
        3.26714434e-023, 9.99977792e-001, 1.05961220e-007],
       [1.09261113e-019, 4.55690808e-066, 4.55155340e-025, ...,
        3.04807610e-028, 9.99999899e-001, 4.01081489e-009],
       [1.37622871e-006, 2.04910996e-050, 1.74320976e-006, ...,
        6.69459576e-007, 3.20817487e-107, 7.77815500e-005]])